In [5]:
import streamlit as st
import streamlit.components.v1 as components
import csv
import string
import os
import pickle
import pandas as pd
from datetime import datetime as dt
from base64 import b64encode
#import proximitypyhash as pph
#import pygeohash as gh
from streamlit_folium import st_folium
from streamlit_folium import folium_static
import folium
import numpy as np

from agile.filtering import query_location, query_date, query_adid, query_node
from agile.mapping import data_map 
from agile.locations import locations_of_interest
from agile.people import colocation
from agile.prediction import double_cluster, get_top_N_clusters, haversine
from agile.utils.tag import find_all_nearby_nodes
from agile.utils.geocode import reverse_geocode
from agile.utils.files import find, random_line, save, random_name
from agile.utils.dataframes import modify_and_sort_columns
from agile.profile import Profile
from agile.samsreport import Report
from agile.centrality import compute_top_centrality
from agile.overview import adid_value_counts

from streamlit_option_menu import option_menu
import pygeohash as gh

In [6]:
df = pd.read_csv('data/weeklong_gh.csv')

In [7]:
df['datetime'] = pd.to_datetime(df['datetime'],errors='coerce')

In [8]:
df = modify_and_sort_columns(df)

In [9]:
df

,advertiser_id,advertiser_id_alias,Unnamed: 0.1,datetime,latitude,longitude,geohash
0,f6e83770-ae29-2381-57af-19232d2980d1,None,0,2018-04-15 00:00:00,46.253140,-119.140060,c25r0uhu56
1,82753786-15c1-46c5-4303-770fa8802eb3,None,1,2018-04-15 00:00:00,46.262040,-119.410247,c25p4v8pgt
2,53578967-974f-1664-85d8-851d3b215a81,None,2,2018-04-15 00:00:00,46.406660,-119.287900,c270j814qk
3,387ef50d-2500-60d3-6f53-10379b778422,None,3,2018-04-15 00:00:00,46.236100,-119.240400,c25pn95bc3
4,95175906-1466-5d7a-25d3-2d693d706928,None,4,2018-04-15 00:00:00,46.241551,-119.142671,c25r0f4bw9
...,...,...,...,...,...,...,...
7684447,7645323d-3d80-f127-9f49-5e72fce6ef95,None,7684447,2018-04-08 23:59:59,46.255700,-119.259577,c25pnhtffr
7684448,ba00512e-2711-4108-5323-64023001dfe7,None,7684448,2018-04-08 23:59:59,46.276267,-119.281460,c25pm8mfu9
7684449,33026704-1230-4780-78bf-091ab7811618,None,7684449,2018-04-08 23:59:59,46.212900,-119.106504,c25qcsv39k
7684450,e08cb98c-8dc8-b890-3638-6d870b5647d6,None,7684450,2018-04-08 23:59:59,46.263570,-119.398550,c25p5n08u4


In [13]:
len(df['advertiser_id'].unique())

10605

In [34]:
first = []
last = []
name_list = []
with open(find('../names/first.txt')) as F, open(find('../names/last.txt')) as L:
    for line_f in F:
        first.append(line_f.strip())
    for line_l in L:
        last.append(line_l.strip())
        
    for f in first:
        for l in last:        
            name_list.append(l + '-' + f)